In [65]:
import os
import utils
import sys
import math
from collections import Counter

In [15]:
try:
    sys.path.append('/opt/MatterSim/build/')  # local docker or Philly
    import MatterSim
except: 
    # local conda env only
    sys.path.append('/home/hoyeung/Documents/vnla/code/build')  
    import MatterSim

In [22]:
os.environ['PT_DATA_DIR'] = '/home/hoyeung/blob_matterport3d/'
PT_DATA_DIR = '/home/hoyeung/blob_matterport3d/'

In [5]:
# read the training scan names
with open(PT_DATA_DIR + 'semantics/asknav_tr_scans.txt', 'r') as fh:
    tr_scans = fh.read().split('\n')[:-1]

len(tr_scans)

56

In [6]:
# read the room types in training scans
with open(PT_DATA_DIR + 'semantics/asknav_tr_room_types.txt', 'r') as fh:
    room_types = fh.read().split('\n')[:-1]

len(room_types)

30

In [19]:
viewix_heading_elevation_map = {}
for i in range(0,12):
    viewix_heading_elevation_map[i] = (i*math.pi/6, -math.pi/6)
for i in range(12,24):
    viewix_heading_elevation_map[i] = (i*math.pi/6, 0)
for i in range(24,36):
    viewix_heading_elevation_map[i] = (i*math.pi/6, +math.pi/6)

In [18]:
PT_DATA_DIR = '/home/hoyeung/blob_matterport3d/'
sim = MatterSim.Simulator()
sim.setRenderingEnabled(False)
sim.setDiscretizedViewingAngles(True)
sim.setCameraResolution(640, 480)
sim.setCameraVFOV(math.radians(60))
sim.setNavGraphPath(
    os.path.join(PT_DATA_DIR, 'connectivity'))
sim.init()

In [92]:
def _check_facing(closest_vertex):
    if (closest_vertex.rel_heading < math.pi/12.0 and closest_vertex.rel_heading > -math.pi/12.0):
        return True    

In [34]:
# for each scan, get all vertices
# construct [room label][next room label][elevation]

all_pano_to_region = {} # scan: G

scans_list = []
viewpoint_list = []
curr_room_label_list = []
next_room_label_list = []
viewix_list = []
elevation_list = []

for scan in tr_scans:
    
    if scan not in all_pano_to_region.keys():
        all_pano_to_region[scan] = utils.load_panos_to_region(scan, '')
    G = utils.load_nav_graphs(scan)
    
    for n in G.nodes:
        curr_rm_label = all_pano_to_region[scan][n]
        long_id = scan + '_' + n
        
        for viewix in range(36):
            # get any closest vertex
            heading, elevation = viewix_heading_elevation_map[viewix]
            sim.newEpisode(scan, n, heading, elevation)
            state = sim.getState()
            assert state.viewIndex == viewix           
            
            # map to direct facing vertex
            if len(state.navigableLocations) > 1:
                closest_vertex = state.navigableLocations[1]
                if _check_facing(closest_vertex):
                    next_vertexId = closest_vertex.viewpointId
                else:
                    next_vertexId = None
            else:
                next_vertexId = None
            
            if next_vertexId:
                next_rm_label = all_pano_to_region[scan][next_vertexId]
            else:
                next_rm_label = "None"
            
            scans_list.append(scan)
            viewpoint_list.append(n)
            curr_room_label_list.append(curr_rm_label)
            next_room_label_list.append(next_rm_label)
            viewix_list.append(viewix)
            elevation_list.append(viewix // 12)

In [93]:
assert len(scans_list) == len(viewpoint_list) == len(curr_room_label_list) == len(next_room_label_list) == len(viewix_list) == len(elevation_list)
set(curr_room_label_list) == set(room_types)

True

## Next room considered Lookup

In [ ]:
scans_list.append(scan)
viewpoint_list.append(n)
curr_room_label_list.append(curr_rm_label)
next_room_label_list.append(next_rm_label)
viewix_list.append(viewix)
elevation_list.append(viewix // 12)

In [113]:
lookup_from_curr_and_next_rm_typ_and_elevation = {room_type:{room_type:{0:[], 1:[], 2:[]} for room_type in room_types + ['None']} for room_type in room_types}

In [114]:
for i in range(len(curr_room_label_list)):
    lookup_from_curr_and_next_rm_typ_and_elevation[curr_room_label_list[i]][next_room_label_list[i]][elevation_list[i]].append((scans_list[i], viewpoint_list[i], viewix_list[i]))

#### Analyze the distribution a bit

In [125]:
cts_per_curr_and_next_rm_typ_and_elevation = []
keys = []
for curr_rm in room_types:
    for next_rm in room_types + ['None']:
        keys.append((curr_rm, next_rm))
        cts_per_curr_and_next_rm_typ_and_elevation.append(len(lookup_from_curr_and_next_rm_typ_and_elevation[curr_rm][next_rm][0]))

In [123]:
len(cts_per_curr_and_next_rm_typ_and_elevation)

930

In [126]:
keys

[('-', '-'),
 ('-', 'B'),
 ('-', 'C'),
 ('-', 'S'),
 ('-', 'Z'),
 ('-', 'a'),
 ('-', 'b'),
 ('-', 'c'),
 ('-', 'd'),
 ('-', 'e'),
 ('-', 'f'),
 ('-', 'g'),
 ('-', 'h'),
 ('-', 'i'),
 ('-', 'j'),
 ('-', 'k'),
 ('-', 'l'),
 ('-', 'm'),
 ('-', 'n'),
 ('-', 'o'),
 ('-', 'p'),
 ('-', 'r'),
 ('-', 's'),
 ('-', 't'),
 ('-', 'u'),
 ('-', 'v'),
 ('-', 'w'),
 ('-', 'x'),
 ('-', 'y'),
 ('-', 'z'),
 ('-', 'None'),
 ('B', '-'),
 ('B', 'B'),
 ('B', 'C'),
 ('B', 'S'),
 ('B', 'Z'),
 ('B', 'a'),
 ('B', 'b'),
 ('B', 'c'),
 ('B', 'd'),
 ('B', 'e'),
 ('B', 'f'),
 ('B', 'g'),
 ('B', 'h'),
 ('B', 'i'),
 ('B', 'j'),
 ('B', 'k'),
 ('B', 'l'),
 ('B', 'm'),
 ('B', 'n'),
 ('B', 'o'),
 ('B', 'p'),
 ('B', 'r'),
 ('B', 's'),
 ('B', 't'),
 ('B', 'u'),
 ('B', 'v'),
 ('B', 'w'),
 ('B', 'x'),
 ('B', 'y'),
 ('B', 'z'),
 ('B', 'None'),
 ('C', '-'),
 ('C', 'B'),
 ('C', 'C'),
 ('C', 'S'),
 ('C', 'Z'),
 ('C', 'a'),
 ('C', 'b'),
 ('C', 'c'),
 ('C', 'd'),
 ('C', 'e'),
 ('C', 'f'),
 ('C', 'g'),
 ('C', 'h'),
 ('C', 'i'),
 ('C',

In [127]:
# very sparse
list(zip(keys, cts_per_curr_and_next_rm_typ_and_elevation))

[(('-', '-'), 270),
 (('-', 'B'), 0),
 (('-', 'C'), 0),
 (('-', 'S'), 22),
 (('-', 'Z'), 2),
 (('-', 'a'), 72),
 (('-', 'b'), 140),
 (('-', 'c'), 8),
 (('-', 'd'), 44),
 (('-', 'e'), 51),
 (('-', 'f'), 24),
 (('-', 'g'), 0),
 (('-', 'h'), 201),
 (('-', 'i'), 1),
 (('-', 'j'), 5),
 (('-', 'k'), 27),
 (('-', 'l'), 58),
 (('-', 'm'), 8),
 (('-', 'n'), 16),
 (('-', 'o'), 20),
 (('-', 'p'), 56),
 (('-', 'r'), 11),
 (('-', 's'), 50),
 (('-', 't'), 2),
 (('-', 'u'), 5),
 (('-', 'v'), 5),
 (('-', 'w'), 11),
 (('-', 'x'), 0),
 (('-', 'y'), 49),
 (('-', 'z'), 6),
 (('-', 'None'), 2844),
 (('B', '-'), 0),
 (('B', 'B'), 2),
 (('B', 'C'), 0),
 (('B', 'S'), 0),
 (('B', 'Z'), 0),
 (('B', 'a'), 0),
 (('B', 'b'), 0),
 (('B', 'c'), 0),
 (('B', 'd'), 1),
 (('B', 'e'), 0),
 (('B', 'f'), 0),
 (('B', 'g'), 0),
 (('B', 'h'), 3),
 (('B', 'i'), 0),
 (('B', 'j'), 0),
 (('B', 'k'), 0),
 (('B', 'l'), 0),
 (('B', 'm'), 0),
 (('B', 'n'), 0),
 (('B', 'o'), 0),
 (('B', 'p'), 0),
 (('B', 'r'), 0),
 (('B', 's'), 0),
 (

In [128]:
import pickle
import json
with open(PT_DATA_DIR + 'semantics/asknav_curr_next_rmtyp_ele_to_image.json', 'w') as fh:
    json.dump(lookup_from_curr_and_next_rm_typ_and_elevation, fh)

In [129]:
with open(PT_DATA_DIR + 'semantics/asknav_curr_next_rmtyp_ele_to_image.json', 'r') as fh:
    reload_lookup = json.load(fh)

In [131]:
reload_lookup['p']['f']

{'0': [['1pXnuDYAj8r', '5aef41e3b2994baa813ed2450f7b17c7', 9],
  ['p5wJjkQkbXX', '601e588142ad4be893ca588dd7608fb8', 7],
  ['p5wJjkQkbXX', '0e088291a0cd4b0295d77345cd21ecbc', 8],
  ['p5wJjkQkbXX', '0e088291a0cd4b0295d77345cd21ecbc', 10],
  ['p5wJjkQkbXX', '6854cd178a4641b5b8386ad0121b1b04', 10],
  ['p5wJjkQkbXX', '6854cd178a4641b5b8386ad0121b1b04', 11],
  ['b8cTxDM8gDG', '5d45547e87c14406bd8785e7cce66162', 0],
  ['b8cTxDM8gDG', 'f2944e0b66b9461994a7f757582f9bc3', 10],
  ['b8cTxDM8gDG', '886b3d659bac40eb8462cc1af42c8d87', 1],
  ['b8cTxDM8gDG', '886b3d659bac40eb8462cc1af42c8d87', 2],
  ['5LpN3gDmAk7', 'aa5bf4cd5d0448bf8be5f5dbc0b53fe3', 4],
  ['5LpN3gDmAk7', 'aa5bf4cd5d0448bf8be5f5dbc0b53fe3', 5],
  ['5LpN3gDmAk7', '7dca42b9f8a04177b4766e6b90a76aef', 6],
  ['e9zR4mvMWw7', '782cb05e39a14434bdd09ff295c0354a', 9],
  ['e9zR4mvMWw7', '782cb05e39a14434bdd09ff295c0354a', 10],
  ['rPc6DW4iMge', '877ddff288064faeb2e0b5107bd6a0bc', 1],
  ['rPc6DW4iMge', 'e5d088fa8d764d6daa2e8daf7c9eb035', 0],
  ['

## Simple lookup (no consideration of next room connection)

In [46]:
lookup_from_curr_rm_typ_and_elevation = {room_type:{0:[], 1:[], 2:[]} for room_type in room_types}

for i in range(len(curr_room_label_list)):
    lookup_from_curr_rm_typ_and_elevation[curr_room_label_list[i]][elevation_list[i]].append((scans_list[i], viewpoint_list[i], viewix_list[i]))
    

In [77]:
lookup_from_curr_rm_typ_and_elevation['u']

{0: [('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 0),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 1),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 2),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 3),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 4),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 5),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 6),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 7),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 8),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 9),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 10),
  ('ac26ZMwG7aT', '312c145aacbd4014909e033462abdd9e', 11),
  ('ac26ZMwG7aT', 'fb6bdce12ffa4d87a740d226ae62c38c', 0),
  ('ac26ZMwG7aT', 'fb6bdce12ffa4d87a740d226ae62c38c', 1),
  ('ac26ZMwG7aT', 'fb6bdce12ffa4d87a740d226ae62c38c', 2),
  ('ac26ZMwG7aT', 'fb6bdce12ffa4d87a740d226ae62c38c', 3),
  ('ac26ZMwG7aT', 'fb6bdce12ffa4d87a740d226ae62c38c', 4),
  ('ac26Z

In [79]:
import pickle
import json
with open(PT_DATA_DIR + 'semantics/asknav_rmtyp_ele_to_image.json', 'w') as fh:
    json.dump(lookup_from_curr_rm_typ_and_elevation, fh)

In [80]:
with open(PT_DATA_DIR + 'semantics/asknav_rmtyp_ele_to_image.json', 'r') as fh:
    reload_lookup = json.load(fh)

#### Analyze the distribution a bit

In [55]:
cts_per_rm_typ_and_elevation = []
for rm in room_types:
    cts_per_rm_typ_and_elevation.append(len(lookup_from_curr_rm_typ_and_elevation[rm][0]))
    
list(zip(room_types, cts_per_rm_typ_and_elevation))

[('-', 4008),
 ('B', 24),
 ('C', 216),
 ('S', 1056),
 ('Z', 12),
 ('a', 7512),
 ('b', 11772),
 ('c', 1392),
 ('d', 3756),
 ('e', 3468),
 ('f', 3420),
 ('g', 636),
 ('h', 14424),
 ('i', 768),
 ('j', 564),
 ('k', 4860),
 ('l', 4428),
 ('m', 2364),
 ('n', 2088),
 ('o', 2448),
 ('p', 5472),
 ('r', 1188),
 ('s', 4368),
 ('t', 300),
 ('u', 432),
 ('v', 984),
 ('w', 564),
 ('x', 24),
 ('y', 2448),
 ('z', 2136)]

In [73]:
# for each room type, do a counter to count scans
rm_to_counter = {}
for rm in room_types:
    list_of_tuples = lookup_from_curr_rm_typ_and_elevation[rm][0]
    scans = [tup[0] for tup in list_of_tuples]
    rm_to_counter[rm] = Counter(scans).most_common()

In [75]:
rm_to_counter['-']

[('mJXqzFtmKg4', 360),
 ('ULsKaCPVFJR', 216),
 ('SN83YJsR3w2', 192),
 ('5LpN3gDmAk7', 192),
 ('7y3sRwLe3Va', 192),
 ('kEZ7cmS4wCh', 180),
 ('sKLMLpTHeUy', 180),
 ('qoiz87JEwZ2', 168),
 ('D7N2EKCX4Sj', 156),
 ('82sE5b5pLXE', 144),
 ('b8cTxDM8gDG', 144),
 ('gTV8FGcVJC9', 144),
 ('ur6pFq6Qu1A', 144),
 ('PX4nDJXEHrG', 132),
 ('r1Q1Z4BcV1o', 120),
 ('cV4RVeZvu5T', 108),
 ('aayBHfsNo7d', 84),
 ('XcA2TqTSSAj', 72),
 ('e9zR4mvMWw7', 72),
 ('2n8kARJN3HM', 60),
 ('29hnd4uzFmX', 60),
 ('JmbYfDe2QKZ', 60),
 ('Uxmj2M2itWa', 48),
 ('B6ByNegPMKs', 48),
 ('V2XKFyX4ASd', 48),
 ('EDJbREhghzL', 48),
 ('p5wJjkQkbXX', 48),
 ('S9hNv5qa7GM', 48),
 ('17DRP5sb8fy', 36),
 ('uNb9QFRL6hY', 36),
 ('r47D5H71a5s', 36),
 ('5q7pvUzZiYa', 36),
 ('jh4fc5c5qoQ', 36),
 ('E9uDoFAP3SH', 36),
 ('VzqfbhrpDEA', 36),
 ('ac26ZMwG7aT', 36),
 ('HxpKQynjfin', 36),
 ('i5noydFURQK', 24),
 ('1LXtFkjw3qL', 24),
 ('Vvot9Ly1tCj', 24),
 ('1pXnuDYAj8r', 24),
 ('VVfe2KiqLaN', 24),
 ('sT4fr6TAbpF', 24),
 ('Pm6F8kyY3z2', 12),
 ('pRbA3pwrgk9',